# Santa 2025 - Optimization Hunt (Notebook)

Este notebook faz uma busca (hunt) para tentar melhorar o score do seu `BASE_SUBMISSION` (calculado abaixo) usando uma estratégia prática:

1. **Hunt em N=200** com `santa_packing._tools.hunt_compact_contact` (múltiplos seeds + jobs) e polish opcional (`bin/post_opt`).
2. **Mother prefix**: gera soluções para N=1..199 pegando o prefixo da solução de N=200.
3. **Pós-processamento**: `santa_packing.cli.improve_submission` com varredura de `--smooth-window` + `--improve-n200`.
4. **Validação Kaggle** (`overlap_mode="kaggle"`) e **autofix** apenas se necessário.

Tudo roda dentro do notebook (sem `subprocess` para CLIs Python).


In [1]:
from __future__ import annotations

import os
import sys
import time
from pathlib import Path

# --- SETUP PATHS (run from repo root or notebooks/) ---
current_path = Path.cwd().resolve()
if (current_path / "santa_packing").exists():
    project_root = current_path
elif (current_path.parent / "santa_packing").exists():
    project_root = current_path.parent
else:
    raise RuntimeError("Could not find repo root (missing santa_packing/)")

os.chdir(project_root)
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

print(f"Project root: {project_root}")

# --- Meta / objetivo ---
# `CURRENT_BEST` é calculado automaticamente a partir do BASE_SUBMISSION (com OVERLAP_MODE).
CURRENT_BEST = None

# Pare cedo quando atingir este score (use None para desativar)
TARGET_SCORE = 69.999999  # objetivo: ficar < 70

# --- Main knobs ---
NMAX = 200
JOBS = max(1, (os.cpu_count() or 16) - 2)
OVERLAP_MODE = "strict"  # strict | conservative | kaggle

# Coloque aqui o seu CSV (pode ser absoluto), ex:
# BASE_SUBMISSION = Path("/caminho/para/submission_70_78.csv")
BASE_SUBMISSION = Path("submission.csv")

# Se o BASE_SUBMISSION tiver overlap em OVERLAP_MODE, escolha o que fazer:
# - "solve": gera um baseline kaggle-safe via `santa_packing.workflow.solve` (mais lento)
# - "mother_prefix": tenta reparar via mother-prefix (rápido, mas pode piorar score)
# - "autofix": roda autofix completo (muito lento)
# - "abort": para com erro
BASE_INVALID_ACTION = "solve"
BASE_INVALID_SOLVE_CONFIG = Path("configs/submit_strong.json")

RUN_TAG = time.strftime("%Y%m%d_%H%M%S")
RUN_DIR = Path("runs") / f"notebook_optimization_hunt_{RUN_TAG}"
RUN_DIR.mkdir(parents=True, exist_ok=True)

print(f"JOBS={JOBS}")
print(f"OVERLAP_MODE={OVERLAP_MODE}")
print(f"RUN_DIR={RUN_DIR}")


Project root: /home/marcux777/Santa-2025-Christmas-Tree-Packing-Challenge
JOBS=14
OVERLAP_MODE=kaggle
RUN_DIR=runs/notebook_optimization_hunt_20260111_103707


## 0. Utilitários
Funções auxiliares para rodar o pipeline (sem subprocess), score Kaggle, gerar prefixos e registrar o melhor candidato.


In [2]:
import csv
import contextlib
import io
import json
import numpy as np
from dataclasses import dataclass
from pathlib import Path


def score_csv(csv_path: Path, *, overlap_mode: str, check_overlap: bool = True) -> float:
    from santa_packing.scoring import score_submission

    res = score_submission(
        Path(csv_path),
        nmax=int(NMAX),
        check_overlap=bool(check_overlap),
        overlap_mode=str(overlap_mode),
        require_complete=True,
    )
    return float(res.score)


def try_score(csv_path: Path) -> tuple[bool, float | None, str | None]:
    try:
        s = score_csv(csv_path, overlap_mode=str(OVERLAP_MODE), check_overlap=True)
        return True, s, None
    except Exception as e:
        return False, None, str(e)


def _call_main(main_func, argv: list[str]) -> None:
    try:
        rc = int(main_func(argv))
    except SystemExit as e:
        rc = int(e.code or 1)
    if rc != 0:
        name = getattr(main_func, "__module__", "main")
        raise RuntimeError(f"{name} failed (rc={rc})")


def autofix(inp: Path, out: Path, *, seed: int = 123) -> Path:
    from santa_packing.cli.autofix_submission import main as autofix_main

    out.parent.mkdir(parents=True, exist_ok=True)
    argv = [
        str(inp),
        "--out",
        str(out),
        "--nmax",
        str(int(NMAX)),
        "--overlap-mode",
        str(OVERLAP_MODE),
        "--seed",
        str(int(seed)),
    ]
    _call_main(autofix_main, argv)
    return out


def detouch_fast(inp: Path, out: Path, *, max_scale: float = 1.02) -> Path:
    """Fast Kaggle-safe detouch (scale-only). If it can't fix, it raises."""
    from santa_packing.scoring import first_overlap_pair, load_submission
    from santa_packing.submission_format import fit_xy_in_bounds, quantize_for_submission
    from santa_packing.cli.improve_submission import _write_submission
    from santa_packing.tree_data import TREE_POINTS

    out.parent.mkdir(parents=True, exist_ok=True)
    points = np.array(TREE_POINTS, dtype=float)
    puzzles = load_submission(inp, nmax=int(NMAX))
    fixed: dict[int, np.ndarray] = {}

    def _try_scale(points: np.ndarray, poses: np.ndarray, scale: float) -> np.ndarray | None:
        cand = np.array(poses, dtype=float, copy=True)
        center = np.mean(cand[:, 0:2], axis=0)
        cand[:, 0:2] = center[None, :] + (cand[:, 0:2] - center[None, :]) * float(scale)
        cand = fit_xy_in_bounds(cand)
        cand = quantize_for_submission(cand)
        if first_overlap_pair(points, cand, mode=str(OVERLAP_MODE)) is None:
            return cand
        return None

    for n in range(1, int(NMAX) + 1):
        poses = puzzles.get(n)
        if poses is None or poses.shape != (n, 3):
            raise RuntimeError(f"Missing puzzle {n} or wrong shape {None if poses is None else poses.shape}")

        poses = fit_xy_in_bounds(poses)
        poses = quantize_for_submission(poses)
        if first_overlap_pair(points, poses, mode=str(OVERLAP_MODE)) is None:
            fixed[n] = poses
            continue

        hi = float(max_scale)
        cand_hi = _try_scale(points, poses, hi)
        if cand_hi is None:
            raise RuntimeError(f"fast detouch failed for puzzle {n} (need > max_scale={max_scale})")

        lo = 1.0
        best = hi
        for _ in range(24):
            mid = 0.5 * (lo + best)
            if _try_scale(points, poses, mid) is not None:
                best = mid
            else:
                lo = mid

        fixed[n] = _try_scale(points, poses, best) or cand_hi

    _write_submission(out, fixed, nmax=int(NMAX))
    return out


def improve_submission(inp: Path, out: Path, *, smooth_window: int, improve_n200: bool = True) -> Path:
    from santa_packing.cli.improve_submission import main as improve_main

    out.parent.mkdir(parents=True, exist_ok=True)
    argv = [
        str(inp),
        "--out",
        str(out),
        "--nmax",
        str(int(NMAX)),
        "--smooth-window",
        str(int(smooth_window)),
        "--overlap-mode",
        str(OVERLAP_MODE),
    ]
    if bool(improve_n200) and int(NMAX) >= 200:
        argv.append("--improve-n200")
        # Extra knobs (safe defaults if not defined in the config cell)
        insert_centers = int(globals().get("N200_INSERT_CENTERS", 4000))
        insert_angles = int(globals().get("N200_INSERT_ANGLES", 20))
        insert_pad = float(globals().get("N200_INSERT_PAD_SCALE", 0.15))
        sa_batch = int(globals().get("N200_SA_BATCH", 32))
        sa_steps = int(globals().get("N200_SA_STEPS", 5000))
        argv += [
            "--n200-insert-centers",
            str(insert_centers),
            "--n200-insert-angles",
            str(insert_angles),
            "--n200-insert-pad-scale",
            str(insert_pad),
            "--n200-sa-batch",
            str(sa_batch),
            "--n200-sa-steps",
            str(sa_steps),
        ]

    try:
        _call_main(improve_main, argv)
    except Exception as e:
        if "--improve-n200" not in argv:
            raise
        print(f"[warn] improve_submission failed with --improve-n200; retrying without it: {e}")
        argv2 = [tok for tok in argv if tok != "--improve-n200"]
        _call_main(improve_main, argv2)
    return out


def generate_mother_prefix(mother_csv: Path, out_csv: Path, *, reorder: str = "radial") -> Path:
    # Build submission N=1..NMAX by prefix-slicing puzzle NMAX (mother layout).
    # Important: repair/finalize the mother (NMAX) packing first so ALL prefixes are overlap-free.
    from santa_packing.cli.generate_submission import _finalize_puzzle
    from santa_packing.cli.generate_submission import _radial_reorder
    from santa_packing.cli.improve_submission import _write_submission
    from santa_packing.geom_np import polygon_bbox, transform_polygon
    from santa_packing.tree_data import TREE_POINTS

    def _parse_val(v: str) -> float:
        v = v.strip()
        if v[:1] in ("s", "S"):
            v = v[1:]
        return float(v)

    mother_rows: list[tuple[int, float, float, float]] = []
    with mother_csv.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            n_str, i_str = row["id"].split("_", 1)
            if int(n_str) != int(NMAX):
                continue
            mother_rows.append(
                (
                    int(i_str),
                    _parse_val(row["x"]),
                    _parse_val(row["y"]),
                    _parse_val(row["deg"]),
                )
            )

    mother_rows.sort(key=lambda t: t[0])
    if len(mother_rows) != int(NMAX):
        raise RuntimeError(f"Expected {NMAX} rows for puzzle {NMAX}, got {len(mother_rows)}")

    poses = np.array([[x, y, deg] for (_i, x, y, deg) in mother_rows], dtype=float)
    points = np.array(TREE_POINTS, dtype=float)
    poses = _finalize_puzzle(points, poses, seed=123, puzzle_n=int(NMAX), overlap_mode=str(OVERLAP_MODE))

    reorder_key = str(reorder).lower().strip()
    if reorder_key == "radial":
        poses = _radial_reorder(points, poses)
    elif reorder_key == "greedybbox":
        # Greedy ordering that minimizes prefix AABB growth (fast, no JAX needed).
        bboxes = np.array([polygon_bbox(transform_polygon(points, pose)) for pose in poses], dtype=float)
        single = np.maximum(bboxes[:, 2] - bboxes[:, 0], bboxes[:, 3] - bboxes[:, 1])
        start = int(np.argmin(single))

        order: list[int] = [start]
        remaining = set(range(int(NMAX)))
        remaining.remove(start)
        min_x, min_y, max_x, max_y = (float(x) for x in bboxes[start])

        while remaining:
            best_idx = None
            best_side = None
            best_bbox = None
            for idx in remaining:
                b = bboxes[int(idx)]
                nx = min(min_x, float(b[0]))
                ny = min(min_y, float(b[1]))
                mx = max(max_x, float(b[2]))
                my = max(max_y, float(b[3]))
                side = max(mx - nx, my - ny)
                if best_side is None or side < best_side - 1e-12:
                    best_side = float(side)
                    best_idx = int(idx)
                    best_bbox = (nx, ny, mx, my)
            assert best_idx is not None and best_bbox is not None
            order.append(best_idx)
            remaining.remove(best_idx)
            min_x, min_y, max_x, max_y = best_bbox

        poses = poses[np.array(order, dtype=int)]
    elif reorder_key in {"none", ""}:
        pass
    else:
        raise ValueError(f"Unknown reorder={reorder!r} (use 'radial', 'greedybbox', or 'none')")

    puzzles = {n: np.array(poses[:n], dtype=float, copy=True) for n in range(1, int(NMAX) + 1)}
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    _write_submission(out_csv, puzzles, nmax=int(NMAX))
    return out_csv


def run_hunt_compact_contact(
    *,
    base: Path,
    out_dir: Path,
    seeds: str,
    target_range: str,
    passes: int,
    attempts_per_pass: int,
    post_opt: bool,
    post_iters: int,
    post_restarts: int,
) -> Path:
    from santa_packing._tools.hunt_compact_contact import main as hunt_main

    out_dir.mkdir(parents=True, exist_ok=True)

    argv = [
        "--base",
        str(base),
        "--out-dir",
        str(out_dir),
        "--seeds",
        str(seeds),
        "--jobs",
        str(int(JOBS)),
        "--nmax",
        str(int(NMAX)),
        "--target-range",
        str(target_range),
        "--passes",
        str(int(passes)),
        "--attempts-per-pass",
        str(int(attempts_per_pass)),
        "--ensemble-out",
        str(out_dir / "ensemble.csv"),
        "--choices-out",
        str(out_dir / "ensemble_choices.csv"),
    ]

    if bool(post_opt):
        argv += [
            "--post-opt",
            "--post-iters",
            str(int(post_iters)),
            "--post-restarts",
            str(int(post_restarts)),
            "--post-overlap-mode",
            str(OVERLAP_MODE),
        ]

    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        try:
            rc = int(hunt_main(argv))
        except SystemExit as e:
            rc = int(e.code or 1)
    if rc != 0:
        raise RuntimeError(f"hunt_compact_contact failed (rc={rc})")

    last = [ln.strip() for ln in buf.getvalue().splitlines() if ln.strip()][-1]
    return Path(last)


@dataclass(frozen=True)
class Candidate:
    label: str
    csv_path: Path
    score: float


def save_leaderboard(rows: list[Candidate], path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        json.dump(
            [
                {
                    "label": r.label,
                    "csv_path": str(r.csv_path),
                    "score": r.score,
                }
                for r in sorted(rows, key=lambda x: x.score)
            ],
            f,
            indent=2,
        )


# --- Validate BASE_SUBMISSION once (keep a kaggle-safe base for the hunt loop) ---
BASE_SUBMISSION_RAW = Path(BASE_SUBMISSION)
BASE_WAS_INVALID = False
if not BASE_SUBMISSION_RAW.is_file():
    raise FileNotFoundError(f"BASE_SUBMISSION not found: {BASE_SUBMISSION_RAW}")

base_ok, base_score, base_err = try_score(BASE_SUBMISSION_RAW)
if base_ok:
    BASE_SUBMISSION = BASE_SUBMISSION_RAW
else:
    BASE_WAS_INVALID = True
    print(f"[warn] BASE_SUBMISSION tem overlap ({base_err}).")
    print(f"[info] Vou criar um baseline válido (kaggle-safe) via BASE_INVALID_ACTION={BASE_INVALID_ACTION!r}.")

    action = str(BASE_INVALID_ACTION).lower().strip()
    if action == "mother_prefix":
        mother_fixed = RUN_DIR / "base_mother_prefix.csv"
        mother_fixed = generate_mother_prefix(BASE_SUBMISSION_RAW, mother_fixed)
        base_ok, base_score, base_err = try_score(mother_fixed)
        if not base_ok:
            raise RuntimeError(f"mother-prefix ainda inválido: {base_err}")
        BASE_SUBMISSION = mother_fixed
    elif action == "autofix":
        fixed = RUN_DIR / "base_autofixed.csv"
        fixed = autofix(BASE_SUBMISSION_RAW, fixed, seed=123)
        base_ok, base_score, base_err = try_score(fixed)
        if not base_ok:
            raise RuntimeError(f"autofix ainda inválido: {base_err}")
        BASE_SUBMISSION = fixed
    elif action == "solve":
        from santa_packing.workflow import solve

        cfg = Path(BASE_INVALID_SOLVE_CONFIG)
        if not cfg.is_file():
            raise FileNotFoundError(f"Config não encontrado: {cfg}")

        res = solve(
            nmax=int(NMAX),
            overlap_mode=str(OVERLAP_MODE),
            config=cfg,
            improve=True,
            smooth_window=60,
            improve_n200=True,
            autofix=True,
            export=False,
        )
        base_ok, base_score, base_err = try_score(res.final_submission)
        if not base_ok:
            raise RuntimeError(f"solve() produziu um CSV inválido: {base_err}")
        BASE_SUBMISSION = res.final_submission
    elif action == "abort":
        raise RuntimeError(f"BASE_SUBMISSION inválido: {base_err}")
    else:
        raise ValueError(f"Unknown BASE_INVALID_ACTION={BASE_INVALID_ACTION!r}")

CURRENT_BEST = float(base_score)
print(f"BASE_SUBMISSION_RAW: {BASE_SUBMISSION_RAW}")
print(f"BASE_SUBMISSION (valid): {BASE_SUBMISSION}")
print(f"BASE_SCORE: {CURRENT_BEST:.12f} (overlap_mode={OVERLAP_MODE})")


## 1. Configurar o Hunt
Sugestão: rode por blocos de seeds (ex: 4000..4127) e acumule resultados no `RUN_DIR`.

Dica: comece em `MODE="quick"` para validar o pipeline; depois aumente seeds/passes/iters.


In [3]:
MODE = "target70"  # quick | full | target70

# Estratégia do hunt:
# - best: usa o melhor atual (kaggle-safe)
# - base: usa o BASE_SUBMISSION válido
# - raw: usa o BASE_SUBMISSION_RAW (mesmo que tenha overlap; útil se seu CSV é bom mas inválido)
HUNT_BASE_MODE = "raw" if bool(globals().get("BASE_WAS_INVALID", False)) else "best"  # best | base | raw

TRY_DIRECT = True  # usa o output direto do hunt (soluções independentes por n)
USE_MOTHER_PREFIX = True  # também tenta mother-prefix (nested) a partir do NMAX
MOTHER_REORDERS = ["radial", "greedybbox"]  # radial | greedybbox | none

# Para ficar <70 você precisa melhorar o score completo (1..200), não só N=200.
TARGET_RANGE = "1,200"  # ex: "200,200" (só N=200) ou "1,200" (pipeline completo)

# Seeds em blocos (o notebook continua gerando blocos até atingir TARGET_SCORE)
SEED_START = 4000
SEED_BLOCK_SIZE = 16
MAX_BLOCKS = 1 if MODE == "quick" else None  # None = roda até você parar/atingir TARGET_SCORE

if MODE == "quick":
    PASSES = 600
    ATTEMPTS_PER_PASS = 200
    POST_OPT = False
    POST_ITERS = 20_000
    POST_RESTARTS = 2
    SMOOTH_WINDOWS = [0, 60]
    IMPROVE_N200_OPTIONS = [False]
elif MODE == "full":
    PASSES = 5000
    ATTEMPTS_PER_PASS = 500
    POST_OPT = True
    POST_ITERS = 200_000
    POST_RESTARTS = 32
    SMOOTH_WINDOWS = [0, 60, 120, 180, 199]
    IMPROVE_N200_OPTIONS = [False, True]
else:  # target70
    PASSES = 8000
    ATTEMPTS_PER_PASS = 800
    POST_OPT = True
    POST_ITERS = 600_000
    POST_RESTARTS = 64
    SMOOTH_WINDOWS = [0, 60, 80, 100, 120, 140, 160, 180, 199]
    IMPROVE_N200_OPTIONS = [False, True]

# Blend/repair: tenta aproveitar candidatos com overlap sem rodar `autofix_submission` inteiro.
# Ideia: primeiro faz blend barato (apenas puzzles já válidos), depois tenta reparar (finalize) só os TOPK candidatos.
BLEND_REPAIR_TOPK = 0 if MODE == "quick" else 6
BLEND_REPAIR_MODE = "none" if MODE == "quick" else "finalize"  # none | finalize
BLEND_REPAIR_MAX_PUZZLES = 0 if MODE == "quick" else 25
BLEND_REPAIR_MIN_GAIN = 0.0000  # ganho mínimo estimado em (s^2/n) para tentar repair
BLEND_REPAIR_N_MIN = 1
BLEND_REPAIR_N_MAX = None  # ex: 200

# Knobs do improve_n200 (só tem efeito se JAX estiver instalado)
if MODE == "quick":
    N200_INSERT_CENTERS = 6000
    N200_INSERT_ANGLES = 30
    N200_INSERT_PAD_SCALE = 0.20
    N200_SA_BATCH = 32
    N200_SA_STEPS = 8000
else:
    N200_INSERT_CENTERS = 20000
    N200_INSERT_ANGLES = 60
    N200_INSERT_PAD_SCALE = 0.20
    N200_SA_BATCH = 64
    N200_SA_STEPS = 50000

print({
    "MODE": MODE,
    "HUNT_BASE_MODE": HUNT_BASE_MODE,
    "TRY_DIRECT": TRY_DIRECT,
    "SEED_START": SEED_START,
    "SEED_BLOCK_SIZE": SEED_BLOCK_SIZE,
    "MAX_BLOCKS": MAX_BLOCKS,
    "TARGET_RANGE": TARGET_RANGE,
    "PASSES": PASSES,
    "ATTEMPTS_PER_PASS": ATTEMPTS_PER_PASS,
    "POST_OPT": POST_OPT,
    "POST_ITERS": POST_ITERS,
    "POST_RESTARTS": POST_RESTARTS,
    "SMOOTH_WINDOWS": SMOOTH_WINDOWS,
    "IMPROVE_N200_OPTIONS": IMPROVE_N200_OPTIONS,
    "USE_MOTHER_PREFIX": USE_MOTHER_PREFIX,
    "MOTHER_REORDERS": MOTHER_REORDERS,
    "BLEND_REPAIR_TOPK": BLEND_REPAIR_TOPK,
    "BLEND_REPAIR_MODE": BLEND_REPAIR_MODE,
    "BLEND_REPAIR_MAX_PUZZLES": BLEND_REPAIR_MAX_PUZZLES,
    "BLEND_REPAIR_MIN_GAIN": BLEND_REPAIR_MIN_GAIN,
    "BLEND_REPAIR_N_MIN": BLEND_REPAIR_N_MIN,
    "BLEND_REPAIR_N_MAX": BLEND_REPAIR_N_MAX,
    "N200_INSERT_CENTERS": N200_INSERT_CENTERS,
    "N200_INSERT_ANGLES": N200_INSERT_ANGLES,
    "N200_INSERT_PAD_SCALE": N200_INSERT_PAD_SCALE,
    "N200_SA_BATCH": N200_SA_BATCH,
    "N200_SA_STEPS": N200_SA_STEPS,
})


{'MODE': 'full', 'SEED_BLOCKS': ['4000..4015', '4016..4031', '4032..4047', '4048..4063', '4064..4079', '4080..4095', '4096..4111', '4112..4127'], 'PASSES': 5000, 'ATTEMPTS_PER_PASS': 500, 'POST_OPT': True, 'POST_ITERS': 200000, 'POST_RESTARTS': 64, 'SMOOTH_WINDOWS': [0, 60, 80, 100, 120, 140, 160, 180, 199], 'IMPROVE_N200_OPTIONS': [False, True]}


## 2. Rodar Hunt + Pós-processamento + Seleção
Este loop:
- roda `hunt_compact_contact` no intervalo `TARGET_RANGE` (por default `1,200` para melhorar o score completo)
- gera o submission por prefixo a partir da solução de N=200
- aplica `improve_submission` (smoothing + improve-n200)
- valida/score em modo Kaggle
- mantém o melhor candidato


In [ ]:
if CURRENT_BEST is None:
    raise RuntimeError("CURRENT_BEST is None (run the setup cells first)")

from santa_packing.blend import blend_submissions

best: Candidate = Candidate(label="BASE", csv_path=BASE_SUBMISSION, score=float(CURRENT_BEST))
all_rows: list[Candidate] = [best]
seed_blocks_run: list[str] = []
stop = False

seed = int(SEED_START)
blocks_run = 0
while True:
    if MAX_BLOCKS is not None and blocks_run >= int(MAX_BLOCKS):
        print(f"[stop] reached MAX_BLOCKS={MAX_BLOCKS}")
        break

    block = f"{seed}..{seed + int(SEED_BLOCK_SIZE) - 1}"
    seed_blocks_run.append(block)
    seed += int(SEED_BLOCK_SIZE)
    blocks_run += 1

    block_dir = RUN_DIR / f"hunt_seeds_{block.replace('..','-').replace(',','_')}"

    hmode = str(HUNT_BASE_MODE).lower().strip()
    if hmode == "best":
        hunt_base = best.csv_path
    elif hmode == "raw":
        hunt_base = BASE_SUBMISSION_RAW
    else:  # base
        hunt_base = BASE_SUBMISSION

    hunt_out = run_hunt_compact_contact(
        base=hunt_base,
        out_dir=block_dir,
        seeds=block,
        target_range=TARGET_RANGE,
        passes=PASSES,
        attempts_per_pass=ATTEMPTS_PER_PASS,
        post_opt=POST_OPT,
        post_iters=POST_ITERS,
        post_restarts=POST_RESTARTS,
    )

    base_candidates: list[tuple[str, Path]] = []
    if bool(TRY_DIRECT):
        base_candidates.append(("direct", hunt_out))

    if bool(USE_MOTHER_PREFIX):
        for reorder in list(MOTHER_REORDERS):
            mother_tag = f"mother_{str(reorder).lower()}"
            mother_out = block_dir / f"submission_mother_prefix_{str(reorder).lower()}.csv"
            mother_out = generate_mother_prefix(hunt_out, mother_out, reorder=str(reorder))
            base_candidates.append((mother_tag, mother_out))

    if not base_candidates:
        raise RuntimeError("No base strategy enabled (set TRY_DIRECT and/or USE_MOTHER_PREFIX).")

    # 1) Gerar candidatos (smoothing + n200) e tentar blend barato (apenas puzzles já válidos)
    ranked: list[tuple[float, str, Path]] = []  # (score_no_check, label, path)
    for base_label, base_for_post in base_candidates:
        for w in SMOOTH_WINDOWS:
            for improve_n200 in IMPROVE_N200_OPTIONS:
                tag = "n200" if bool(improve_n200) else "no_n200"
                label = f"{block}|{base_label}|smooth={w}|{tag}"
                improved = block_dir / f"submission_improved_{base_label}_smooth{int(w)}_{tag}.csv"
                improved = improve_submission(base_for_post, improved, smooth_window=int(w), improve_n200=bool(improve_n200))

                no_check = score_csv(improved, overlap_mode=str(OVERLAP_MODE), check_overlap=False)
                ranked.append((float(no_check), label, improved))

                blended = block_dir / f"submission_blend_{base_label}_smooth{int(w)}_{tag}.csv"
                blended, meta = blend_submissions(
                    base_csv=best.csv_path,
                    candidate_csv=improved,
                    out_csv=blended,
                    nmax=int(NMAX),
                    overlap_mode=str(OVERLAP_MODE),
                    repair_mode="none",
                )

                if int(meta.get("used_candidate", 0)) <= 0:
                    continue

                ok, s, err = try_score(blended)
                if not ok:
                    print(f"[skip] blend inválido: {err}")
                    continue

                row = Candidate(label=label, csv_path=blended, score=float(s))
                all_rows.append(row)

                if row.score < best.score:
                    best = row
                    print(f"\n[new best] {best.score:.12f}  ({best.label}) -> {best.csv_path}")
                    if TARGET_SCORE is not None and best.score <= float(TARGET_SCORE):
                        print(f"[done] reached TARGET_SCORE={TARGET_SCORE}")
                        stop = True
                        break

            if stop:
                break
        if stop:
            break

    if stop:
        break

    # 2) Repair (bounded): tenta reparar apenas TOPK candidatos mais promissores
    if int(BLEND_REPAIR_TOPK) > 0 and str(BLEND_REPAIR_MODE).lower().strip() != "none":
        ranked.sort(key=lambda t: t[0])
        for rank, (_no_check, label, cand_path) in enumerate(ranked[: int(BLEND_REPAIR_TOPK)], start=1):
            slug = label.replace("..", "-").replace("|", "_")
            out_path = block_dir / f"submission_blend_repair{rank}_{slug}.csv"
            out_path, meta = blend_submissions(
                base_csv=best.csv_path,
                candidate_csv=cand_path,
                out_csv=out_path,
                nmax=int(NMAX),
                overlap_mode=str(OVERLAP_MODE),
                repair_mode=str(BLEND_REPAIR_MODE).lower().strip(),
                repair_seed=123,
                repair_max_puzzles=int(BLEND_REPAIR_MAX_PUZZLES),
                repair_min_gain=float(BLEND_REPAIR_MIN_GAIN),
                repair_n_min=int(BLEND_REPAIR_N_MIN),
                repair_n_max=None if BLEND_REPAIR_N_MAX is None else int(BLEND_REPAIR_N_MAX),
            )

            if int(meta.get("used_candidate", 0)) <= 0:
                continue

            ok, s, err = try_score(out_path)
            if not ok:
                print(f"[skip] repair inválido: {err}")
                continue

            row = Candidate(label=f"{label}|repair_top{rank}", csv_path=out_path, score=float(s))
            all_rows.append(row)

            if row.score < best.score:
                best = row
                print(f"\n[new best] {best.score:.12f}  ({best.label}) -> {best.csv_path}")
                if TARGET_SCORE is not None and best.score <= float(TARGET_SCORE):
                    print(f"[done] reached TARGET_SCORE={TARGET_SCORE}")
                    stop = True
                    break

    if stop:
        break

save_leaderboard(all_rows, RUN_DIR / "leaderboard.json")
print(f"\nTried {len(all_rows)} candidates")
print(f"Best: {best}")
print(f"Target to beat: {CURRENT_BEST}")


Running compact_contact: seeds=16 jobs=14 nmax=200 target=200..200
[seed=4002] score=72.807999675374 time=102.2s
[seed=4005] score=72.807999675374 time=107.1s
[seed=4008] score=72.807999675374 time=108.1s
[seed=4001] score=72.807999675374 time=108.6s
[seed=4012] score=72.807999675374 time=109.4s
[seed=4009] score=72.807999675374 time=109.5s
[seed=4011] score=72.807999675374 time=110.4s
[seed=4006] score=72.807999675374 time=111.3s
[seed=4010] score=72.807999675374 time=112.0s
[seed=4013] score=72.807999675374 time=113.0s
[seed=4003] score=72.807999675374 time=114.8s
[seed=4004] score=72.807999675374 time=114.9s
[seed=4000] score=72.807999675374 time=115.4s
[seed=4007] score=72.807999675374 time=119.6s
[seed=4014] score=72.807999675374 time=62.4s
[seed=4015] score=72.807999675374 time=60.8s
Best single: seed=4002 score=72.807999675374 csv=runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/cc_seed4002.csv
Ensemble score (no overlap check): 72.807999675374
Running post_o

Post-opt complete
Initial score: 72.807999675
Final score:   72.801975641
Phase1 improved: 87
Backprop improved: 0
Elapsed: 1130.838000000s
Saved: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/ensemble_postopt_raw.csv


[warn] Post-opt blend failed; keeping previous (runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/ensemble.csv): Base CSV has overlap in puzzle 4 (mode=strict).
[warn] Post-opt output still overlaps (strict): Overlap detected in puzzle 4
Post-opt score (strict, no-overlap): 72.807999675374


wrote: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth0_no_n200.csv
score(no-overlap): 256.303049961088
s_max: 8.280944

[new best] 256.303049961088  (4000..4015|smooth=0|no_n200) -> runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth0_no_n200.csv


wrote: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth0_n200.csv
score(no-overlap): 256.304870955503
s_max: 8.302905
wrote: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth60_no_n200.csv
score(no-overlap): 104.847051115612
s_max: 8.280944

[new best] 104.847051115612  (4000..4015|smooth=60|no_n200) -> runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth60_no_n200.csv
wrote: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth60_n200.csv
score(no-overlap): 104.848487138023
s_max: 8.298267
wrote: runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth80_no_n200.csv
score(no-overlap): 92.332812118904
s_max: 8.280944

[new best] 92.332812118904  (4000..4015|smooth=80|no_n200) -> runs/notebook_optimization_hunt_20260111_103707/hunt_seeds_4000-4015/submission_improved_smooth

## 3. Finalizar
Exporta o melhor CSV para `submission.csv` (raiz) e cria um bundle em `submissions/`.


In [ ]:
from santa_packing.workflow import archive_submission

if best is None:
    raise RuntimeError("No valid candidates produced")

res = archive_submission(
    best.csv_path,
    nmax=int(NMAX),
    overlap_mode=str(OVERLAP_MODE),
    name=f"notebook_hunt_{RUN_TAG}",
    export=True,
    export_path=project_root / "submission.csv",
    extra_meta={
        "notebook": "notebooks/02_optimization_hunt.ipynb",
        "run_dir": str(RUN_DIR),
        "mode": MODE,
        "base_submission_raw": str(BASE_SUBMISSION_RAW),
        "base_submission_valid": str(BASE_SUBMISSION),
        "base_was_invalid": bool(BASE_WAS_INVALID),
        "base_invalid_action": str(BASE_INVALID_ACTION),
        "seed_blocks": list(seed_blocks_run),
        "seed_start": int(SEED_START),
        "seed_block_size": int(SEED_BLOCK_SIZE),
        "max_blocks": MAX_BLOCKS,
        "hunt_base_mode": str(HUNT_BASE_MODE),
        "target_range": str(TARGET_RANGE),
        "smooth_windows": list(SMOOTH_WINDOWS),
        "improve_n200_options": list(IMPROVE_N200_OPTIONS),
        "try_direct": bool(TRY_DIRECT),
        "use_mother_prefix": bool(USE_MOTHER_PREFIX),
        "mother_reorders": list(MOTHER_REORDERS),
        "blend_repair_topk": int(BLEND_REPAIR_TOPK),
        "blend_repair_mode": str(BLEND_REPAIR_MODE),
        "blend_repair_max_puzzles": int(BLEND_REPAIR_MAX_PUZZLES),
        "blend_repair_min_gain": float(BLEND_REPAIR_MIN_GAIN),
        "blend_repair_n_min": int(BLEND_REPAIR_N_MIN),
        "blend_repair_n_max": None if BLEND_REPAIR_N_MAX is None else int(BLEND_REPAIR_N_MAX),
        "best_label": best.label,
        "best_path": str(best.csv_path),
        "best_score_observed": float(best.score),
    },
)

print(f"Archived: {res.run_dir}")
print(f"Exported: {res.exported_submission}")
print(f"Score: {res.score:.12f} (overlap_mode={OVERLAP_MODE})")
